In [1]:
import requests
import time
import pandas as pd
from tqdm import tqdm
import os
from glob import glob
import json

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
region = 'kr'
columns = ['summonerId', 'summonerName', 'leaguePoints', 'rank', 'wins', \
           'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak']

api_key = 'your_api_key'

In [ ]:
tiers = ['grandmasterleagues', 'challengerleagues']
df = pd.DataFrame(columns=columns)

for tier in tiers:
    url = f'https://{region}.api.riotgames.com/lol/league/v4/{tier}/by-queue/RANKED_SOLO_5x5?api_key={api_key}'
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    entries = data['entries']

    for entrie in entries:
        df = pd.concat([df, pd.DataFrame([entrie])])

In [ ]:
df = df.reset_index(drop=True)
df.to_csv('top_tier.csv', index=False)
df.head()

In [ ]:
api_keys = [
    'your_api_key',
    'your_api_key',
    'your_api_key',
    'your_api_key',
    'your_api_key',
    'your_api_key',
    'your_api_key',
]

In [ ]:
api_key_index = 0

for i in tqdm(range(0, len(df))):
    summonerName = df.iloc[i]['summonerName']
    api_key = api_keys[api_key_index]

    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}?api_key={api_key}'
    
    response = requests.get(url)
    try:
        response.raise_for_status()
    except:
        if response.status_code != 404:
            print(i, summonerName, url)
            break
        continue

    data = response.json()
    ppuid = data['puuid']

    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{ppuid}/ids?type=ranked&start=0&count=30&api_key={api_key}'
    response = requests.get(url)
    
    game_list = response.json()

    with open("top_list.txt", "a") as f:
        for game_id in game_list:
            f.write(game_id + "\n")

    api_key_index = (api_key_index + 1) % len(api_keys)

## 정재하기

In [ ]:
with open("top_list.txt", "r") as file:
    lines_seen = set(file.read().splitlines())

with open("top_list2.txt", "w") as file:
    for line in lines_seen:
        file.write(line + '\n')

In [ ]:
len(lines_seen)

## 게임 데이터 분할하기 for json API

In [ ]:
os.makedirs("top_list", exist_ok=True)

In [ ]:
num = 3000

total_lines = len(lines_seen)
num_files = total_lines // num

if total_lines % num != 0: 
    num_files += 1

lines_seen = list(lines_seen)

for i in range(num_files):
    with open(f"top_list/{i+1}.txt", "w") as file:
        for line in lines_seen[i*num:(i+1)*num]:
            file.write(line + '\n')

## 게임 데이터 가져오기

In [4]:
game_pathes = glob("./grand_master/*.json")

games = []
for path in tqdm(game_pathes):
    with open(path, "r") as f:
        data = json.load(f)

    if data['info']['queueId'] == 420:
        games.append(data)

len(game_pathes), len(games)

100%|██████████| 26924/26924 [00:18<00:00, 1435.05it/s]


(26924, 20020)

In [ ]:
tier_df = pd.read_csv('./top_tier.csv')

In [ ]:
for game in games:
    participants = game['info']['participants']
    teams = game['info']['teams']

    win = [1 if team['win'] == True else 0 for team in teams]

    dfs = pd.DataFrame()
    for participant in participants:
        championName = participant['championName']
        # summonerName = participant['summonerName']
        # summonerId = participant['summonerId']

        # summoner = tier_df[tier_df['summonerId'] == summonerId]
        # print(len(summoner))

        summoner = tier_df[tier_df['summonerName'] == participant['summonerName']]
        
        if len(summoner):
            leaguePoints = summoner['leaguePoints'].iloc[0]
            wins = summoner['wins'].iloc[0]
            losses = summoner['losses'].iloc[0]
            leaguePoints = summoner['leaguePoints'].iloc[0]
            hotStreak = summoner['hotStreak'].iloc[0] # 연승
        else:
            leaguePoints = None
            wins = None
            losses = None
            leaguePoints = None
            hotStreak = None
        
        df = pd.DataFrame([[summonerName, championName, leaguePoints, wins, losses, leaguePoints, hotStreak]])
        dfs = pd.concat([dfs, df])

In [ ]:
df = pd.DataFrame()

for game in tqdm(games):
    participants = game['info']['participants']
    teams = game['info']['teams']
    win = [1 if team['win'] == True else 0 for team in teams]

    champions = list()
    for participant in participants:
        championName = participant['championName']
        champions.append(championName)

    champions += win
    
    df = pd.concat([df, pd.DataFrame(champions).transpose()])

df.columns = ["blue_top", "blue_jungle", "blue_middle", "blue_bottom", "blue_utility",
              "red_top", "red_jungle", "red_middle", "red_bottom", "red_utility",
              "blue_win", "red_win"]

df = df.reset_index(drop=True)

## Machine learning

In [ ]:
label_encoder = LabelEncoder()
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = label_encoder.fit_transform(df[column])

X = df.drop(['blue_win', 'red_win'], axis=1)
y = df['blue_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

## 통계적 접근

In [ ]:
op_df = pd.read_csv('./op_gg.csv')

op_df.loc[op_df['line'] == 'top', 'line'] = 'TOP'  # teamPosition
op_df.loc[op_df['line'] == 'jungle', 'line'] = 'JUNGLE'
op_df.loc[op_df['line'] == 'mid', 'line'] = 'MIDDLE'
op_df.loc[op_df['line'] == 'adc', 'line'] = 'BOTTOM'
op_df.loc[op_df['line'] == 'support', 'line'] = 'UTILITY'

In [ ]:
champions = set()

acc = 0
for game in tqdm(games):
    participants = game['info']['participants']
    teams = game['info']['teams']
    win = int(teams[0]['win'])

    tiers = [3] * 10

    for i, participant in enumerate(participants):
        championName = participant['championName']
        teamPosition = participant['teamPosition']
        
        try:
            tier = op_df.loc[(op_df['Champion'] == championName) & (op_df['line'] == teamPosition), 'tier'].iloc[0]
        except:
            tier = 3
    
        tiers[i] = tier

    pred = int(sum(tiers[:5]) < sum(tiers[5:]))
    acc += int(win == pred)

In [ ]:
print(acc / len(games))